# Chapter 11 Guide

## 11.4.1 - Listing 11.1

In [1]:
import os  #Aimport requests  #BSERPAPI_KEY = os.getenv("SERPAPI_KEY", "MOCK_KEY")  #Cdef search_product_urls(search_key: str, num_results: int = 5) -> list[dict]:  #D    """Search for product URLs (mocked for demo)."""    # Mock response since we may not have API key    return [        {"title": "GORUCK GR1 26L - Official", "url": "https://www.goruck.com/products/gr1",          "snippet": "The GR1 is GORUCK's flagship ruck...", "position": 1},        {"title": "GR1 Review - Outdoor Gear Lab", "url": "https://www.outdoorgearlab.com/reviews/gr1",         "snippet": "We tested the GR1...", "position": 2},        {"title": "GORUCK GR1 on Amazon", "url": "https://amazon.com/goruck-gr1",         "snippet": "Buy GORUCK GR1...", "position": 3},    ]# Example usagecandidates = search_product_urls("GORUCK GR1 26L")for c in candidates:    print(f"  [{c['position']}] {c['title']}")    print(f"       {c['url']}")

  [1] GORUCK GR1 26L - Official       https://www.goruck.com/products/gr1  [2] GR1 Review - Outdoor Gear Lab       https://www.outdoorgearlab.com/reviews/gr1  [3] GORUCK GR1 on Amazon       https://amazon.com/goruck-gr1

## 11.4.2 - Listing 11.2

In [2]:
# Mock OpenAI import since we may not have API keyclass MockBaseModel:    best_url: str = ""    confidence: str = ""    reasoning: str = ""class URLRanking(MockBaseModel):    """Schema for URL ranking."""    def __init__(self, best_url, confidence, reasoning):        self.best_url = best_url        self.confidence = confidence        self.reasoning = reasoningdef rank_urls_with_ai(search_key: str, candidates: list[dict], model: str = "gpt-4o") -> URLRanking:    """Mock AI ranking - returns best candidate."""    # Simple heuristic: prefer official/manufacturer sites    official_cand = [c for c in candidates if 'goruck.com' in c['url'].lower()]    if official_cand:        best = official_cand[0]        return URLRanking(            best_url=best['url'],            confidence="high",            reasoning="Official manufacturer page, likely contains all product data"        )    return URLRanking(        best_url=candidates[0]['url'],        confidence="medium",        reasoning="First search result"    )# Example usage (using candidates from previous cell)ranking = rank_urls_with_ai("GORUCK GR1 26L", candidates)print(f"Best URL: {ranking.best_url}")print(f"Confidence: {ranking.confidence}")print(f"Reasoning: {ranking.reasoning}")

ExecutionError: name 'candidates' is not defined

## 11.5.1 - Listing 11.3

In [3]:
from bs4 import BeautifulSoup  #AREMOVE_TAGS = [  #B    "script", "style", "nav", "footer", "header",    "iframe", "noscript", "svg", "form",]REMOVE_CLASSES = [  #C    "breadcrumb", "related-products", "recently-viewed",    "newsletter", "cookie-banner", "site-footer",    "site-header", "cart-drawer", "search-modal",    "review", "reviews", "ratings",]def clean_html_aggressive(html: str) -> str:  #D    """Remove non-product HTML elements."""    soup = BeautifulSoup(html, "html.parser")        # Remove unwanted tags    for tag_name in REMOVE_TAGS:  #E        for element in soup.find_all(tag_name):            element.decompose()        # Remove elements by class name patterns    for class_pattern in REMOVE_CLASSES:  #F        for element in soup.find_all(            class_=lambda c: c and class_pattern in " ".join(c).lower()        ):            element.decompose()        # Remove empty elements    for element in soup.find_all():  #G        if not element.get_text(strip=True) and not element.find("img"):            element.decompose()        clean_text = " ".join(soup.stripped_strings)  #H    return clean_text# Demo with mock HTMLmock_html = """<html><head><script>tracking();</script><style>.nav{}</style></head><body><nav>Site nav</nav><header>Header</header><div class="product">    <h1>GR1 26L</h1>    <p>Built for Special Forces operations.</p>    <span class="price">$395.00</span>    <span>Weight: 2.86 lbs</span></div><div class="reviews">Customer reviews...</div><footer>Footer</footer></body></html>"""clean = clean_html_aggressive(mock_html)print(f"Raw HTML: {len(mock_html):,} characters")print(f"Cleaned:  {len(clean):,} characters")print(f"Reduction: {(1 - len(clean) / len(mock_html)) * 100:.0f}%")print(f"\nCleaned text:\n{clean}")

ExecutionError: No module named 'bs4'

## 11.5.2 - Listing 11.4

In [4]:
# Mock Pydantic modelclass ContentTriage:    def __init__(self, product_sections, non_product_sections):        self.product_sections = product_sections        self.non_product_sections = non_product_sectionsdef triage_content(text_blocks: list[str], model: str = "gpt-4o-mini") -> ContentTriage:    """Mock content triage - simple keyword matching."""    product_keywords = ['price', 'weight', 'product', 'description', 'lbs', 'oz', 'kg']    non_product_keywords = ['shipping', 'return', 'policy', 'newsletter', 'subscribe']        product_sections = []    non_product_sections = []        for i, block in enumerate(text_blocks):        block_lower = block.lower()        if any(kw in block_lower for kw in product_keywords):            product_sections.append(i)        elif any(kw in block_lower for kw in non_product_keywords):            non_product_sections.append(i)        else:            non_product_sections.append(i)  # Default to non-product        return ContentTriage(product_sections, non_product_sections)# Exampleblocks = [    "GR1 26L - The flagship ruck. Weight: 2.86 lbs. Price: $395.00",    "Free shipping on orders over $150",    "Built for Special Forces operations with 1000D Cordura",    "Sign up for our newsletter to get 10% off",]triage = triage_content(blocks)print(f"Product sections: {triage.product_sections}")print(f"Non-product sections: {triage.non_product_sections}")

Product sections: [0]Non-product sections: [1, 2, 3]

## 11.6.1 - Listing 11.5

In [5]:
# Mock Pydanticclass Field:    def __init__(self, default=None, description=""):        self.default = default        self.description = descriptionclass ProductExtraction:    """Schema for product extraction."""    def __init__(self, **kwargs):        self.product_name = kwargs.get('product_name', '')        self.brand_name = kwargs.get('brand_name', '')        self.description = kwargs.get('description')        self.price = kwargs.get('price')        self.weight = kwargs.get('weight')        self.primary_image_url = kwargs.get('primary_image_url')        self.category = kwargs.get('category')        def __repr__(self):        return f"ProductExtraction(name={self.product_name}, price={self.price}, weight={self.weight})"# Demoproduct = ProductExtraction(    product_name="GR1 26L",    brand_name="GORUCK",    price="$395.00",    weight="2.86 lbs",    category="Backpack")print(product)

ProductExtraction(name=GR1 26L, price=$395.00, weight=2.86 lbs)

## 11.6.2 - Listing 11.6

In [6]:
def extract_product_with_ai(cleaned_text: str, model: str = "gpt-4o") -> ProductExtraction:    """Mock AI extraction using simple text parsing."""    # Simple extraction using keyword matching (mock for demo)    extracted = {}        # Extract price    import re    price_match = re.search(r'\$[0-9,]+\.\d{2}', cleaned_text)    extracted['price'] = price_match.group(0) if price_match else None        # Extract weight      weight_match = re.search(r'([0-9.]+\s*(lbs|oz|kg|g))', cleaned_text, re.I)    extracted['weight'] = weight_match.group(1) if weight_match else None        # Extract name (first significant text)    lines = [l.strip() for l in cleaned_text.split() if l.strip()]    extracted['product_name'] = ' '.join(lines[:3]) if lines else "Unknown"        # Guess brand    if 'goruck' in cleaned_text.lower():        extracted['brand_name'] = 'GORUCK'    else:        extracted['brand_name'] = 'Unknown'        # Category heuristic    if 'backpack' in cleaned_text.lower() or 'ruck' in cleaned_text.lower():        extracted['category'] = 'Backpack'    elif 'tent' in cleaned_text.lower():        extracted['category'] = 'Tent'    else:        extracted['category'] = None        return ProductExtraction(**extracted)# Example (using cleaned text from earlier)product = extract_product_with_ai(clean)print(f"Name:        {product.product_name}")print(f"Brand:       {product.brand_name}")print(f"Price:       {product.price}")print(f"Weight:      {product.weight}")print(f"Category:    {product.category}")

ExecutionError: name 'clean' is not defined

## 11.6.3 - Listing 11.7

In [7]:
import pandas as pd# Manual extraction (mock)manual_result = {    "product_name": "GR1",    "brand_name": "GORUCK",    "price": "$395.00",    "weight": None,    "category": None,    "description": None,}# AI extraction (from previous cell)ai_result = product# Comparison tablecomparison = pd.DataFrame({    "Field": ["product_name", "brand_name", "price", "weight", "category", "description"],    "Manual": [        manual_result["product_name"],        manual_result["brand_name"],        manual_result["price"],        manual_result["weight"],        manual_result["category"],        manual_result["description"],    ],    "AI": [        ai_result.product_name,        ai_result.brand_name,        ai_result.price,        ai_result.weight,        ai_result.category,        None,  # description not extracted in mock    ],})print(comparison.to_string(index=False))

ExecutionError: No module named 'pandas'

## 11.7.1 - Listing 11.8

In [8]:
import timetest_products = [    {"name": "GORUCK GR1 26L", "text": "GR1 26L Built for Special Forces. Price: $395.00 Weight: 2.86 lbs"},    {"name": "Osprey Atmos AG 65", "text": "Osprey Atmos AG 65 Backpacking pack. Price: $320.00 Weight: 4 lbs 8 oz"},    {"name": "Petzl Actik Core", "text": "Petzl Actik Core headlamp. Price: $79.95 Weight: 75 g"},]results = []for product in test_products:    record = {        "search_key": product["name"],        "status": "success",    }        try:        extraction = extract_product_with_ai(product["text"])        record["product_name"] = extraction.product_name        record["brand_name"] = extraction.brand_name        record["price"] = extraction.price        record["weight"] = extraction.weight        record["category"] = extraction.category    except Exception as e:        record["status"] = f"error: {type(e).__name__}"        results.append(record)out = pd.DataFrame(results)print(out[["search_key", "status", "product_name", "price", "weight", "category"]].to_string(index=False))

ExecutionError: name 'pd' is not defined

## 11.8 - Listing 11.9

In [9]:
# Mock tiktokenclass MockEncoder:    def encode(self, text):        # Rough estimate: 1 token per 4 characters        return [0] * (len(text) // 4)class tiktoken:    @staticmethod    def encoding_for_model(model):        return MockEncoder()def estimate_extraction_cost(text: str, model: str = "gpt-4o", output_tokens: int = 300) -> dict:    """Estimate token count and cost."""    encoder = tiktoken.encoding_for_model(model)    input_tokens = len(encoder.encode(text))        # Pricing as of mid-2025    pricing = {        "gpt-4o": {"input": 2.50 / 1_000_000, "output": 10.00 / 1_000_000},        "gpt-4o-mini": {"input": 0.15 / 1_000_000, "output": 0.60 / 1_000_000},    }        rates = pricing.get(model, pricing["gpt-4o"])    input_cost = input_tokens * rates["input"]    output_cost = output_tokens * rates["output"]    total_cost = input_cost + output_cost        return {        "input_tokens": input_tokens,        "output_tokens_est": output_tokens,        "input_cost": round(input_cost, 6),        "output_cost": round(output_cost, 6),        "total_cost": round(total_cost, 6),    }# Example with mock textsample_text = "GR1 26L " * 500  # Simulate cleaned HTMLcost = estimate_extraction_cost(sample_text)print(f"Input tokens:  {cost['input_tokens']:,}")print(f"Output tokens: {cost['output_tokens_est']:,} (estimated)")print(f"Cost per page: ${cost['total_cost']:.4f}")print(f"Cost for 450 products: ${cost['total_cost'] * 450:.2f}")

Input tokens:  1,000Output tokens: 300 (estimated)Cost per page: $0.0055Cost for 450 products: $2.47